## Model training and tuning for the Industrials sector

In [107]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, MeanAbsoluteError, MeanAbsolutePercentageError
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_evaluation import evaluate
from sktime.split import SlidingWindowSplitter, temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import ForecastingPipeline, TransformedTargetForecaster
from sktime.transformations.series.detrend import Detrender, Deseasonalizer
from sktime.transformations.series.boxcox import LogTransformer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.pipeline import make_pipeline
from sktime.datatypes._panel._convert import from_multi_index_to_nested
from sktime.forecasting.statsforecast import StatsForecastAutoTheta
from sktime.forecasting.arima import AutoARIMA
from sktime.transformations.series.date import DateTimeFeatures
from sktime.forecasting.naive import NaiveVariance

from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index(['ticker', 'date']).sort_index()

def reindex_ticker(group):
    ticker = group.index.get_level_values(0)[0]
    date_range = pd.date_range(start=group.index.get_level_values(1).min(),
                               end=group.index.get_level_values(1).max(),
                               freq='B')
    full_index = pd.MultiIndex.from_product([[ticker], date_range], names=['ticker', 'date'])
    return group.reindex(full_index)

# Apply reindexing by ticker
df_reindexed = df.groupby(level='ticker', group_keys=False).apply(reindex_ticker)

In [3]:
df_industrial = df_reindexed[df_reindexed['GICS Sector'] == 'Industrials']
df_industrial = df_industrial.groupby(level='ticker', group_keys=False).apply(reindex_ticker)

def to_periodindex(df):
    df = df.copy()
    df.index = pd.MultiIndex.from_arrays([
        df.index.get_level_values('ticker'),
        pd.PeriodIndex(df.index.get_level_values('date'), freq='B')
    ], names=['ticker', 'date'])
    return df

df_industrial = to_periodindex(df_industrial)

C:\Users\Home\AppData\Local\Temp\ipykernel_9008\4059970266.py:8: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  pd.PeriodIndex(df.index.get_level_values('date'), freq='B')


In [4]:
df_industrial.isna().sum()

close           0
dividends       0
high            0
low             0
open            0
stock splits    0
volume          0
GICS Sector     0
dtype: int64

In [68]:
transformer = DateTimeFeatures(ts_freq='D')
df_industrial[['year', 'month', 'day']] = transformer.fit_transform(df_industrial[['close']])
# df_industrial[['year', 'month', 'day']] = df_industrial[['year', 'month', 'day']].astype(pd.CategoricalDtype())

d:\investing-bot\.venv\Lib\site-packages\sktime\transformations\series\date.py:448: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  DUMMIES = DUMMIES.replace(
d:\investing-bot\.venv\Lib\site-packages\sktime\transformations\series\date.py:448: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  DUMMIES = DUMMIES.replace(
d:\investing-bot\.venv\Lib\site-packages\sktime\transformations\series\date.py:243: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  x_df["date_sequence"] = time_index.to_times

In [69]:
y_train, y_test, X_train, X_test = temporal_train_test_split(
    y=df_industrial[['close']],
    X=df_industrial[['open', 'volume', 'year', 'month', 'day']],
    test_size=0.1
)

d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  splits = (self._split(pd.Index(inst.values)) for _, inst in srs.groupby(index))
d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  splits = (self._split(pd.Index(inst.values)) for _, inst in srs.groupby(index))
d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  splits = (self._split(pd.Index(inst.values)) for _, inst in srs.groupby(index))
d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated a

## Gradient Boosting

In [ ]:
model = XGBRegressor(
    objective='reg:squarederror',
    random_state=42
)

forecaster = make_reduction(
    model,
    strategy="recursive",
    window_length=30,
    pooling='local'
)

pipeline = TransformedTargetForecaster(
    steps=[
        ("detrender", Detrender(PolynomialTrendForecaster(degree=1))),
        ("deseasonalizer", Deseasonalizer(sp=30)),
        ("forecaster", forecaster)
    ]
)

pipeline.fit(y_train, X_train)

d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])
d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])
d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])
d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])


NaiveVariance(forecaster=TransformedTargetForecaster(steps=[('detrender',
                                                             Detrender(forecaster=PolynomialTrendForecaster())),
                                                            ('deseasonalizer',
                                                             Deseasonalizer(sp=30)),
                                                            ('forecaster',
                                                             RecursiveTabularRegressionForecaster(estimator=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None,...
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...),
                                                                                                  window_length=30))]))

In [109]:
len(y_test.index.get_level_values('date').unique())

261

In [ ]:
test_dates = y_test.index.get_level_values('date').unique()
fh = ForecastingHorizon(test_dates, is_relative=False)
y_pred = pipeline.predict(fh=fh, X=X_test)

d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq 

In [112]:
y_pred

close
ticker date                  
ADP    2024-05-03  239.250670
       2024-05-06  238.923223
       2024-05-07  238.719458
       2024-05-08  240.137767
       2024-05-09  239.659455
...                       ...
XYL    2025-04-28  135.664128
       2025-04-29  134.875103
       2025-04-30  135.061484
       2025-05-01  135.620163
       2025-05-02  136.604045

[18531 rows x 1 columns]

In [113]:
mape = MeanAbsolutePercentageError(symmetric=True)

mape(y_test, y_pred)

0.12067507921528113

## Theta model

In [ ]:
y_train, y_test, X_train, X_test = temporal_train_test_split(
    y=df_industrial[['close']],
    X=df_industrial[['open', 'volume']],
    test_size=0.1
)

d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  splits = (self._split(pd.Index(inst.values)) for _, inst in srs.groupby(index))
d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  splits = (self._split(pd.Index(inst.values)) for _, inst in srs.groupby(index))
d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  splits = (self._split(pd.Index(inst.values)) for _, inst in srs.groupby(index))
d:\investing-bot\.venv\Lib\site-packages\sktime\split\base\_base_splitter.py:189: FutureWarning: PeriodDtype[B] is deprecated a

In [49]:
model = StatsForecastAutoTheta(
    season_length=30
)

model.fit(y_train)

d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])


StatsForecastAutoTheta(season_length=30)

In [50]:
test_dates = y_test.index.get_level_values('date').unique()
fh = ForecastingHorizon(test_dates, is_relative=False)
y_pred = model.predict(fh=fh, X=X_test)

In [51]:
y_pred

close
ticker date                  
ADP    2024-05-03  238.114073
       2024-05-06  237.706678
       2024-05-07  237.749279
       2024-05-08  238.042183
       2024-05-09  238.432462
...                       ...
XYL    2025-04-28  138.608416
       2025-04-29  138.457679
       2025-04-30  138.692953
       2025-05-01  138.478063
       2025-05-02  138.445178

[18531 rows x 1 columns]

In [52]:
mape = MeanAbsolutePercentageError(symmetric=True)

mape(y_test, y_pred)

0.10916706906507993

## Linear Regression

In [53]:
model = LinearRegression()

forecaster = make_reduction(
    model,
    strategy="recursive",
    window_length=30,
    pooling='local'
)

pipeline = make_pipeline(
    LogTransformer(offset=0.1), # offset to avoid log(0)
    Detrender(PolynomialTrendForecaster(degree=1)),
    Deseasonalizer(sp=30),
    forecaster
)
pipeline.fit(y_train, X_train)

d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])
d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])
d:\investing-bot\.venv\Lib\site-packages\sktime\datatypes\_utilities.py:323: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  cuttoff_idx = ensure_index([cutoff])


TransformedTargetForecaster(steps=[LogTransformer(offset=0.1),
                                   Detrender(forecaster=PolynomialTrendForecaster()),
                                   Deseasonalizer(sp=30),
                                   RecursiveTabularRegressionForecaster(estimator=LinearRegression(),
                                                                        window_length=30)])

In [54]:
test_dates = y_test.index.get_level_values('date').unique()
fh = ForecastingHorizon(test_dates, is_relative=False)
y_pred = pipeline.predict(fh=fh, X=X_test)

d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return x + by
d:\investing-bot\.venv\Lib\site-packages\sktime\utils\datetime.py:227: FutureWarning: Period with BDay freq 

In [55]:
y_pred

close
ticker date                  
ADP    2024-05-03  237.119230
       2024-05-06  237.921257
       2024-05-07  238.488898
       2024-05-08  239.727625
       2024-05-09  239.115285
...                       ...
XYL    2025-04-28  125.572605
       2025-04-29  125.408932
       2025-04-30  126.401217
       2025-05-01  126.294870
       2025-05-02  125.906330

[18531 rows x 1 columns]

In [56]:
mape = MeanAbsolutePercentageError(symmetric=True)

mape(y_test, y_pred)

0.10557356024587905